# Overview

The etiqai library enables users to identify and mitigate bias in predictive models. The library is similar to pytorch or sklearn libraries, with a battery of metrics and debiasing algorithms available to find the best debiasing method for the  problem at hand. 

The DataPipeline object holds what we'd like to focus on during the debiasing process: the dataset used for training the model, the model we'd like to test and the fairness metrics used to evaluate results. DebiasPipeline objects take the initial DataPipeline object and apply to it different Identify methods, which aim to generate flags for rows at risk of being biased against, or Repair methods. Repair methods generate new "debiased" datasets. Models trained on debiased datasets perform better on fairness metrics. We have a (growing) collection of repair methods that allow our users to pick the debiased dataset version that best fits their criteria for a good solution. 

# The prediction problem setup

To illustrate some of the library's features, we build a model that predicts whether an applicant makes over or under 50K using the Adult dataset https://archive.ics.uci.edu/ml/datasets/adult. 

In [1]:
from etiq_core import *
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from xgboost.sklearn import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')

Here we're loading the Adult dataset that is already available with the library, but users can use any dataset in a pandas dataframe format for their debiasing analysis. 

In [2]:
data = load_sample('adultdata')

print(data['relationship'].unique())

print(data['marital-status'].unique())

['Own-child' 'Husband' 'Not-in-family' 'Unmarried' 'Wife' 'Other-relative']
['Never-married' 'Married-civ-spouse' 'Widowed' 'Divorced' 'Separated'
 'Married-spouse-absent' 'Married-AF-spouse']


In [3]:
data.head(10)

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K
5,34,Private,198693,10th,6,Never-married,Other-service,Not-in-family,White,Male,0,0,30,United-States,<=50K
6,29,?,227026,HS-grad,9,Never-married,?,Unmarried,Black,Male,0,0,40,United-States,<=50K
7,63,Self-emp-not-inc,104626,Prof-school,15,Married-civ-spouse,Prof-specialty,Husband,White,Male,3103,0,32,United-States,>50K
8,24,Private,369667,Some-college,10,Never-married,Other-service,Unmarried,White,Female,0,0,40,United-States,<=50K
9,55,Private,104996,7th-8th,4,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,10,United-States,<=50K


In [4]:
print('List of columns:', data.columns.values)
print('Nr rows in the dataset:', data.shape[0])

List of columns: ['age' 'workclass' 'fnlwgt' 'education' 'educational-num' 'marital-status'
 'occupation' 'relationship' 'race' 'gender' 'capital-gain' 'capital-loss'
 'hours-per-week' 'native-country' 'income']
Nr rows in the dataset: 48842


## Standard approach to training a binary classifier

We want to predict (classify) if a person's income is above or below 50K using this dataset. Following standard ML practice, after some data cleaning (removing rows with missing values and encoding categorical variables), we split the dataset into train/validate/test groups and train a model for this binary classification task, using the features in the dataset to predict the 'income' variable. 

In [5]:
# data preprocessing

# remove rows with missing values
data = data.replace('?', np.nan)
data.dropna(inplace=True)

# use a LabelEncoder to transform categorical variables 
cont_vars = ['age', 'educational-num', 'fnlwgt', 'capital-gain', 'capital-loss', 'hours-per-week']
cat_vars = list(set(data.columns.values) - set(cont_vars))

label_encoders = {}
data_encoded = pd.DataFrame()
for i in cat_vars:
    label = LabelEncoder()
    data_encoded[i] = label.fit_transform(data[i])
    label_encoders[i] = label
data_encoded.set_index(data.index, inplace=True)
data_encoded = pd.concat([data.loc[:, cont_vars], data_encoded], axis=1).copy()

In [6]:
data_encoded.head(5)

,age,educational-num,fnlwgt,capital-gain,capital-loss,hours-per-week,marital-status,income,relationship,native-country,occupation,gender,education,workclass,race
0,25,7,226802,0,0,40,4,0,3,38,6,1,1,2,2
1,38,9,89814,0,0,50,2,0,0,38,4,1,11,2,4
2,28,12,336951,0,0,40,2,1,0,38,10,1,7,1,4
3,44,10,160323,7688,0,40,2,1,0,38,6,1,15,2,2
5,34,6,198693,0,0,30,4,0,1,38,7,1,0,2,4


In [7]:
# prepare the training/testing/validation datasets

# separate into train/validate/test dataset of sizes 80%/10%/10% as percetages of the initial data
data_remaining, test = train_test_split(data_encoded, test_size=0.1)
train, valid = train_test_split(data_remaining, test_size=0.1112)

# because we don't want to train on protected attributes or labels to be predicted, 
# let's remove these columns from the training dataset
protected_train = train['gender'].copy() # gender is a protected attribute
y_train = train['income'].copy() # labels we're going to train the model to predict
x_train = train.drop(columns=['gender','income'])
protected_valid = valid['gender'].copy() 
y_valid = valid['income'].copy() 
x_valid = valid.drop(columns=['gender','income'])
protected_test = test['gender'].copy() 
y_test = test['income'].copy()
x_test = test.drop(columns=['gender','income'])

In [8]:
# train a XGBoost model to predict 'income'

standard_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=4)    
model_fit = standard_model.fit(x_train, y_train)

How accurate is the model we trained?

In [9]:
y_train_pred = standard_model.predict(x_train)
y_valid_pred = standard_model.predict(x_valid)
print('Model accuracy on the training dataset :', 
      round(100 * accuracy_score(y_train, y_train_pred),2),'%') # round the score to 2 digits  

print('Model accuracy on the validation dataset :', 
      round(100 * accuracy_score(y_valid, y_valid_pred),2),'%')

Model accuracy on the training dataset : 89.95 %
Model accuracy on the validation dataset : 86.24 %


By this point in the analysis, we have a model that is reasonably accurate at predicting whether someone makes more or less than 50K given a few features for that person. But is our model fair?

# Fairness analysis

The etiq library will help us:

1. evaluate how fair is the model we just trained 
2. identify sources of bias 
3. repair the model, which means improve its performance on fairness metrics

The fairness analysis is built around the model we'd like to evaluate, the dataset used to train it and the fairness metrics that are most relevant to our project, which are all wrapped in the DataPipeline object. 

Below, we define the parameters for the debiasing process. What is the protected category? Who is in the privileged and unprivileged groups? What is a positive outcome in this dataset? We include this information in the Dataset class. We define the model we're debiasing (the model we just built) and the fairness metrics we'll use to evaluate model fairness. 

We wrap the model that we want to evaluate for bias, the dataset used to train that model and the metrics we'd like to calculate into a DataPipeline object. 

## Evaluate model performance on fairness metrics

To establish whether the model is fair, we check its performance on fairness metrics. The etiq package has implementations of fairness metrics that are commonly used in the algorithmic fairness literature. Some of the metrics available are : equal opportunity, equal odds, true negative rate, demographic parity. See our documentation for a full list. The user can also declare their own metric that might be more appropriate for their particular problem setup.

We compute the metrics passed on when the DataPipeline object was initialized. Metrics are computed for the training and validation datset, using the model defined in the DataPipeline.  

A commonly used fairness metric is Equal opportunity. The Equal opportunity metric calculates the True Positive Rate for each subgroup of a protected attribute like gender, age, race. A positive prediction from the model is an opportunity, such as "this person makes over 50K, let's approve their mortgage application". With the help of the etiqai library, we can check that the model is just as accurate in predicting who should get that opportunity regardless of gender, race, age. Something to remember is that the Equal opportunity metric accounts for different acceptance rates in the "ground truth" training dataset.

The user can test the performance of different models on the fairness metrics of interest. There are models preloaded in the library or users can use their own model. Below, we use a preloaded logistic regression model and ask for another fairness metric, demographic parity.

In [10]:
# assign the bias parameters: the protected category we'll look at is 'gender'

debias_param = BiasParams(protected='gender', privileged='Male', unprivileged='Female', 
                          positive_outcome_label='>50K', negative_outcome_label='<=50K')

In [11]:
# use etiq_core to apply the same data transformation as shown above in the standard aproach

transforms = [Dropna, EncodeLabels] 

In [12]:
# use the DatasetLoader class to load the original Adult data into the Dataset class and apply transforms
# the DatasetLoader accepts pandas or numpy data structures

dl = DatasetLoader(data=data, label='income', transforms=transforms, bias_params=debias_param,
                   train_valid_test_splits=[0.8, 0.1, 0.1], cat_col=cat_vars,
                   cont_col=cont_vars, names_col = data.columns.values)

In [13]:
# load the model that we'd like to mitigate in tandem with the dataset.
# users can load any model that has a predict function. Here we load a default architecture provided by the library.

xgb = DefaultXGBoostClassifier()
xgb

In [14]:
# specify which metrics we will use to evaluate bias
# the library includes implementations of fairness metrics commonly used in the literature
# users can implement their own metrics

metrics_initial = [accuracy,  equal_opportunity]

In [15]:
# the initial pipeline computes metrics on the loaded dataset and model

pipeline_initial = DataPipeline(dataset_loader=dl, model=xgb, metrics=metrics_initial)
pipeline_initial.run()

INFO:etiq_core.pipeline.DataPipeline852:Starting pipeline
INFO:etiq_core.pipeline.DataPipeline852:Fitting model
INFO:etiq_core.pipeline.DataPipeline852:Computed metrics for the initial dataset
INFO:etiq_core.pipeline.DataPipeline852:Completed pipeline


In [16]:
# check the metrics by group (privileged and unprivileged) for the model trained on the dataset

pipeline_initial.get_protected_metrics()

{'DataPipeline852': [{'accuracy': ('privileged', 0.84, 'unprivileged', 0.93)},
  {'equal_opportunity': ('privileged',
    0.6901408450704225,
    'unprivileged',
    0.55)}]}

Despite high accuracy for the unprivileged group, the xgb model scores lower on equal_opportunity. This results means that more people in the unprivileged group are mistakenly receiving negative outcome labels, when they should be receiving positive outcome labels. 

In [17]:
# the DebiasPipeline aims to identify sources of bias by applying analyses formalized in the Identify pipelines
# the Identify pipeline is looking for 3 sources of bias (limited features, poor sampling and proxies)

identify_pipeline = IdentifyBiasSources(nr_groups=20, # nr of segments based on using unsupervised learning to group similar rows
                                        train_model_segment=True,
                                        group_def=['unsupervised'],
                                        fit_metrics=[accuracy, equal_opportunity])
    
# the DebiasPipeline aims to mitigate sources of bias by applying different types of repair algorithms
# the library offers implementations of repair algorithms described in the academic fairness literature

repair_pipeline = RepairResamplePipeline(steps=[ResampleUnbiasedSegmentsStep(ratio_resample=1)], random_seed=4)

debias_pipeline = DebiasPipeline(data_pipeline=pipeline_initial, 
                                 model=xgb,
                                 metrics=metrics_initial,
                                 identify_pipeline=identify_pipeline,
                                 repair_pipeline=repair_pipeline)
debias_pipeline.run()

INFO:etiq_core.pipeline.DebiasPipeline36:Starting pipeline
INFO:etiq_core.pipeline.DebiasPipeline36:Start Phase IdentifyPipeline844
INFO:etiq_core.pipeline.IdentifyPipeline844:Starting pipeline
INFO:etiq_core.pipeline.IdentifyPipeline844:Completed pipeline
INFO:etiq_core.pipeline.DebiasPipeline36:Completed Phase IdentifyPipeline844
INFO:etiq_core.pipeline.DebiasPipeline36:Start Phase RepairPipeline558
INFO:etiq_core.pipeline.RepairPipeline558:Starting pipeline
INFO:etiq_core.pipeline.RepairPipeline558:Completed pipeline
INFO:etiq_core.pipeline.DebiasPipeline36:Completed Phase RepairPipeline558
INFO:etiq_core.pipeline.DebiasPipeline36:Refitting model
INFO:etiq_core.pipeline.DebiasPipeline36:Computed metrics for the repaired dataset
INFO:etiq_core.pipeline.DebiasPipeline36:Completed pipeline


In [18]:
#change to get_metrics method to get all the metrics
#it needs a bit of an explanation to help user understand what they're looking at (NOTE)

debias_pipeline.get_protected_metrics()

{'DataPipeline852': [{'accuracy': ('privileged', 0.84, 'unprivileged', 0.93)},
  {'equal_opportunity': ('privileged',
    0.6901408450704225,
    'unprivileged',
    0.55)}],
 'DebiasPipeline36': [{'accuracy': ('privileged', 0.82, 'unprivileged', 0.91)},
  {'equal_opportunity': ('privileged',
    0.6539235412474849,
    'unprivileged',
    0.65)}]}

After the repair, the gap between the equal_opportunity metric between privileged and unprivileged groups disappears. Compare equal_opportunity values for the initial pipeline (DataPipeline913) and the debias pipeline (DebiasPipeline983)

In [19]:
# summary of issues identified for each segment

debias_pipeline.get_profiler()

,segment,feature_name,feature_all_statistic,feature_segment_statistic,feature_segment_index,feature_index_ranking,comparison_with_average,group_volume,rank_index,equal_opp_diff,limited_features_issue
254,19,relationship,1.414971,1.420488,100,0,within_avg_interval,2050,12.5,0.150934,no_issue
258,19,hours-per-week,40.996932,41.220488,100,0,within_avg_interval,2050,12.5,0.150934,no_issue
248,19,workclass,2.207873,2.187805,99,1,within_avg_interval,2050,10.5,0.150934,no_issue
250,19,education,10.315734,10.311707,99,1,within_avg_interval,2050,10.5,0.150934,no_issue
251,19,educational-num,10.119278,9.991220,98,2,within_avg_interval,2050,7.5,0.150934,no_issue
...,...,...,...,...,...,...,...,...,...,...,...
4,0,educational-num,10.119278,9.889375,97,3,within_avg_interval,2513,5.0,0.141376,no_issue
12,0,native-country,36.386389,35.532829,97,3,within_avg_interval,2513,5.0,0.141376,no_issue
10,0,capital-loss,88.909332,69.253880,77,23,below_average,2513,3.0,0.141376,no_issue
2,0,fnlwgt,189672.700796,239459.772384,126,26,above_average,2513,2.0,0.141376,no_issue


In [20]:
# two segments

df_test = debias_pipeline.get_profiler()

df_test.loc[(df_test['segment'] == 15) | (df_test['segment'] == 0)]

,segment,feature_name,feature_all_statistic,feature_segment_statistic,feature_segment_index,feature_index_ranking,comparison_with_average,group_volume,rank_index,equal_opp_diff,limited_features_issue
199,15,educational-num,10.119278,10.127098,100,0,within_avg_interval,3218,12.5,0.133722,no_issue
207,15,native-country,36.386389,36.661902,100,0,within_avg_interval,3218,12.5,0.133722,no_issue
198,15,education,10.315734,10.312617,99,1,within_avg_interval,3218,9.5,0.133722,no_issue
200,15,marital-status,2.585775,2.583282,99,1,within_avg_interval,3218,9.5,0.133722,no_issue
202,15,relationship,1.414971,1.438160,101,1,within_avg_interval,3218,9.5,0.133722,no_issue
206,15,hours-per-week,40.996932,40.831262,99,1,within_avg_interval,3218,9.5,0.133722,no_issue
196,15,workclass,2.207873,2.266936,102,2,within_avg_interval,3218,6.0,0.133722,no_issue
201,15,occupation,5.991458,5.910503,98,2,within_avg_interval,3218,6.0,0.133722,no_issue
203,15,race,3.681280,3.627098,98,2,within_avg_interval,3218,6.0,0.133722,no_issue
195,15,age,38.478992,39.909571,103,3,within_avg_interval,3218,4.0,0.133722,no_issue


In [21]:
df_test.loc[(df_test['segment'] == 15) | (df_test['segment'] == 0)].sort_values(['segment','rank_index'],ascending = True).groupby('segment').head(5)

,segment,feature_name,feature_all_statistic,feature_segment_statistic,feature_segment_index,feature_index_ranking,comparison_with_average,group_volume,rank_index,equal_opp_diff,limited_features_issue
9,0,capital-gain,1114.321484,562.797055,50,50,below_average,2513,1.0,0.141376,no_issue
2,0,fnlwgt,189672.700796,239459.772384,126,26,above_average,2513,2.0,0.141376,no_issue
10,0,capital-loss,88.909332,69.253880,77,23,below_average,2513,3.0,0.141376,no_issue
0,0,age,38.478992,37.559491,97,3,within_avg_interval,2513,5.0,0.141376,no_issue
4,0,educational-num,10.119278,9.889375,97,3,within_avg_interval,2513,5.0,0.141376,no_issue
204,15,capital-gain,1114.321484,706.900559,63,37,below_average,3218,1.0,0.133722,no_issue
205,15,capital-loss,88.909332,110.868241,124,24,above_average,3218,2.0,0.133722,no_issue
197,15,fnlwgt,189672.700796,146597.916408,77,23,below_average,3218,3.0,0.133722,no_issue
195,15,age,38.478992,39.909571,103,3,within_avg_interval,3218,4.0,0.133722,no_issue
196,15,workclass,2.207873,2.266936,102,2,within_avg_interval,3218,6.0,0.133722,no_issue


In [22]:
# top 5 features for each segment

debias_pipeline.get_profiler_top5()

,segment,feature_name,feature_segment_statistic,feature_segment_index,comparison_with_average,group_volume
9,0,capital-gain,562.797055,50,below_average,2513
2,0,fnlwgt,239459.772384,126,above_average,2513
10,0,capital-loss,69.253880,77,below_average,2513
0,0,age,37.559491,97,within_avg_interval,2513
4,0,educational-num,9.889375,97,within_avg_interval,2513
...,...,...,...,...,...,...
249,19,fnlwgt,269827.800488,142,above_average,2050
256,19,capital-gain,837.608293,75,below_average,2050
257,19,capital-loss,81.059024,91,within_avg_interval,2050
247,19,age,37.504390,97,within_avg_interval,2050


The debias_pipeline has a repaired dataset. The xbg model trained on the repaired dataset has better equal_opportunity metrics. However, if we'd like to know what issues remain in the repaired datset, we can use the repaired dataset as the input for a new DataPipeline object and apply another identify pipeline like we did above. 

In [23]:
# run Identify methods on top of the repaired dataset
    
evaluate_debias = EvaluateDebiasPipeline(debias_pipeline=debias_pipeline,
                                         identify_pipeline=identify_pipeline)
evaluate_debias.run()

INFO:etiq_core.pipeline.EvaluateDebiasPipeline932:Starting pipeline
INFO:etiq_core.pipeline.DataPipeline257:Starting pipeline
INFO:etiq_core.pipeline.DataPipeline257:Refitting model
INFO:etiq_core.pipeline.DataPipeline257:Computed metrics for the initial dataset
INFO:etiq_core.pipeline.DataPipeline257:Completed pipeline
INFO:etiq_core.pipeline.DebiasPipeline257:Starting pipeline
INFO:etiq_core.pipeline.DebiasPipeline257:Start Phase IdentifyPipeline844
INFO:etiq_core.pipeline.IdentifyPipeline844:Starting pipeline
INFO:etiq_core.pipeline.IdentifyPipeline844:Completed pipeline
INFO:etiq_core.pipeline.DebiasPipeline257:Completed Phase IdentifyPipeline844
INFO:etiq_core.pipeline.DebiasPipeline257:Refitting model
INFO:etiq_core.pipeline.DebiasPipeline257:Computed metrics for the initial dataset
INFO:etiq_core.pipeline.DebiasPipeline257:Completed pipeline
INFO:etiq_core.pipeline.EvaluateDebiasPipeline932:Completed pipeline


In [24]:
evaluate_debias.get_issues_summary_before_repair()

#repaired_dataset.x_train.shape

,issue,features,segments
0,correlation_issue,age,[0]
1,limited_features_issue,N/A,[14]
2,low_unpriv_sample,N/A,"[0, 1, 2, 3, 5, 8, 9, 10, 13, 14, 15, 16, 19]"
4,proxy_issue,relationship,"[0, 1, 2, 3, 5, 8, 9, 10, 13, 14, 15, 16, 19]"
5,skewed_unpriv_sample,N/A,"[1, 3, 5, 10, 14, 15]"
0,low_volume_group,NaN,"[4, 6, 7, 11, 12, 17, 18]"
1,missing_sample,NaN,"[11, 17]"


In [25]:
evaluate_debias.get_issues_summary_after_repair()

,issue,features,segments
0,correlation_issue,capital-loss,[1]
1,correlation_issue,marital-status,[14]
3,proxy_issue,relationship,"[0, 1, 3, 5, 6, 8, 13, 14, 16, 18]"
0,low_volume_group,NaN,"[2, 4, 7, 9, 10, 11, 12, 15, 17, 19]"
1,missing_sample,NaN,"[7, 9, 12, 17, 19]"
